# Sequential 类顺序构建模型

In [1]:
import tensorflow as tf
import keras
import numpy as np

2024-04-08 11:07:17.988270: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 无input,无build模型没有初始化参数，直到调用模型的时候才去初始化参数

In [2]:
# Note that you can also omit the initial `Input`.
# In that case the model doesn't have any weights until the first call
# to a training/evaluation method (since it isn't yet built):
model = keras.Sequential()
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(4))
# model.weights not created yet
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 有Input,无需再build

In [3]:
model = keras.Sequential()
model.add(keras.Input(shape=(16,)))
model.add(keras.layers.Dense(8))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136 (544.00 B)

 Trainable params: 136 (544.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Whereas if you specify an `Input`, the model gets built
# continuously as you are adding layers:
model = keras.Sequential()
model.add(keras.Input(shape=(16,)))
model.add(keras.layers.Dense(8))
len(model.weights)  # Returns "2"
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136 (544.00 B)

 Trainable params: 136 (544.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
# When using the delayed-build pattern (no input shape specified), you can
# choose to manually build your model by calling
# `build(batch_input_shape)`:
model = keras.Sequential()
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(4))
model.build((None, 16))
len(model.weights)  # Returns "4"
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 172 (688.00 B)

 Trainable params: 172 (688.00 B)

 Non-trainable params: 0 (0.00 B)

## 模型运行时build

In [6]:
x = np.random.randint(100, size=(64, 10))
y = np.random.randint(2, size=(64,))
print(x)
print(y)

[[48 25 49  1 98 13  2 36 15 87]
 [49 29 96  2 38 16 14 82 66 33]
 [26 37 46 53 83 22 50 57 32 21]
 [76 89 36 75 65 46 94 32 14 16]
 [35 82 20 36 26 41 52 14  4 63]
 [15 14 25 87 40 97 61 17  7 17]
 [98 71 54 74 88 75 36 78 93 90]
 [68 24 59 24 20 66 25 44  0 77]
 [83  2 31 47 69 35 76 33  2 88]
 [36 14  8 88 85 94  7 76 43  4]
 [ 9 95 54 25 11 84 55 64 64  8]
 [39 18  4 83 41 39 49  8 51 11]
 [61 67 99 71 77 55 37 17 60 95]
 [24 28 32 24 88  7 63 10 49 86]
 [96  4 66 14 91 40 81 73 44 92]
 [41 60 21  7 42 62 52 27 87 11]
 [53 23 64 92 39 65 15 11 86 15]
 [18 29 84 42 23 17 64 39 96 19]
 [22 18 57 89 30 54  7  7 91 74]
 [10 36  9 13 79 80 93 42 92 47]
 [61 23 22 52 23 38 77 10 86 15]
 [ 6  2 46 21 28 94 12 91 10 86]
 [ 6 64  8 49 12 97 48  1 67 16]
 [38 34 58 95 44 81 24 65 78 98]
 [25 52 73 96 85 36 60 94 58 49]
 [78  3 79 95 58 48 54 33 34 78]
 [21 13 30 36 45 15 15 72 15 60]
 [86 89 57 35 20 98 63 60  8 89]
 [95 84 59 63 93 89 35 52 67 49]
 [94 69 21 83 85 34 51 72  7 25]
 [20 83  9

In [7]:
# Note that when using the delayed-build pattern (no input shape specified),
# the model gets built the first time you call `fit`, `eval`, or `predict`,
# or the first time you call the model on some input data.
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=100,output_dim=10))
model.add(keras.layers.Dense(8))
model.add(keras.layers.Dense(1))
model.add(keras.layers.Softmax())
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
history = model.fit(x, y, batch_size=8, epochs=5)

Epoch 1/5


/Users/sunwenjun/anaconda3/envs/python310/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (8, 10, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5722  
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5497 
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5715 
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5763 
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5290 


In [8]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (8, 10, 10)            │         1,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (8, 10, 8)             │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (8, 10, 1)             │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (8, 10, 1)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,099 (4.30 KB)

 Trainable params: 1,097 (4.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [9]:
history